In [1]:
import sys
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
from cvxpy import *
import numpy as np
import math

def unfold(X,n):
    ''' mode-n unfolding of VariableTensor X
    Args: 
        X: input tensor
        n: targeted mode
    Returns:
        matricized version of X
    '''
    shape = X.shape
    ndim = len(shape)
    print ndim
    perm_order = np.roll(np.arange(ndim),n-1)
    X_n= np.reshape(np.transpose(X, perm_order), [shape[n-1],-1])
    return X_n

def norm_tensor(X, p=2):
    '''
    Wrapper function for tensor norm
    '''
    if p == 'nuc_upper':
        # tensor nuclear norm upper bound [FRIEDLAND && Lim 2014]
        expr_list = []
        var_shape = X.size
        var_ndim = len(var_shape)
        for n in range(var_ndim):
            X_n = unfold(X, var_shape, n)
            mask = np.ones(var_ndim, dtype=bool)
            mask[n] = False
            shape_array = np.asarray(var_shape)
            factor = math.sqrt(min(shape_array[mask]))
            norm_X_n = norm(X_n,'nuc')
            norm_X_n = norm_X_n * factor
            expr_list = expr_list + [norm_X_n]
        return min_elemwise(*expr_list)
        
    elif p == 'nuc_lower':
        # tensor nuclear norm lower bound 
        expr_list = []
        for n in range(var_ndim):
            X_n = unfold(X, var_shape, n)
            expr_list = expr_list + [norm_X_n]
        return max_elemwise(*expr_list)

In [2]:

import os
import numpy as np


var_shape = (2,3,4)
mode = 1
X = np.random.random(var_shape)
Omega = np.ones(var_shape)    


import sys
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
from cvxpy import *

X_opt = VariableTensor(var_shape)
# Omega_n = unfold(Omega, var_shape, mode)
X_opt_n = unfold(X_opt, var_shape, mode)


obj = Minimize(norm(X_opt_n))
constraints = [mul_elemwise(Omega, X_opt) == mul_elemwise(Omega, X)]
prob = Problem(obj, constraints)
# Use SCS to solve the problem.
prob.solve(verbose=True, solver=SCS)

mul_elemwise(Omega, X)


NotImplementedError: 

In [ ]:
import sys
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
from cvxpy import *
import numpy as np
n1 = 20;
n2 = 30;
n3 = 10;
X = np.random.random((n1,n2,n3))
X_opt = VariableTensor((n1, n2, n3))
Omega = np.random.choice([0,1], (n1,n2,n3), p = [0.8,0.2])

obj = Minimize(norm_tensor(X_opt, 'nuc_upper'))
constraints = [mul_elemwise(Omega, X_opt) == mul_elemwise(Omega, X)]
prob = Problem(obj, constraints)
# Use SCS to solve the problem.
prob.solve(verbose=True, solver=SCS)

In [ ]:
shape = (2,2,2)
A = np.random.random(shape)
A_out = (A).flatten()
print np.shape(A_out)